In [8]:
import s3fs
import pandas as pd

In [3]:
# Information d'accès au cloud MinIO (Bucket de William)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "williamolivier"

In [ ]:
# Vérification du contenu
fs.ls(f"{MY_BUCKET}/statapp")

In [ ]:
# Récupération des tableaux
FILE_PATH_S3_DCB = f"{MY_BUCKET}/statapp/df_dossier_complet_brut.csv"
with fs.open(FILE_PATH_S3_DCB, "rb") as file:
    df_dossier_complet_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MDCB = f"{MY_BUCKET}/statapp/df_meta_dossier_complet_brut.csv"
with fs.open(FILE_PATH_S3_MDCB, "rb") as file:
    df_meta_dossier_complet_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MB = f"{MY_BUCKET}/statapp/df_mobpro_brut.csv"
with fs.open(FILE_PATH_S3_MB, "rb") as file:
    df_mobpro_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MMB = f"{MY_BUCKET}/statapp/df_meta_mobpro_brut.csv"
with fs.open(FILE_PATH_S3_MMB, "rb") as file:
    df_meta_mobpro_brut = pd.read_csv(file)

In [ ]:
df_mobpro_brut.head()